<a href="https://colab.research.google.com/github/EmreYY20/ToS-Simplification/blob/dev/comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# !pip install PyPDF2
# !pip install nltk
# !pip install transformers
# !pip install rouge


In [60]:
import PyPDF2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import defaultdict
import heapq
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from collections import Counter
from nltk.tokenize import word_tokenize

In [47]:
# PDFs einlesen
def lese_pdf_datei(dateipfad):
    text = ""
    with open(dateipfad, 'rb') as datei:
        pdf_reader = PyPDF2.PdfReader(datei)
        for seite in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[seite].extract_text()  # Verwenden von pages[seite] statt getPage(seite)
    return text

In [62]:
# Extractive
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def extraktive_zusammenfassung(text):
    sentences = sent_tokenize(text)
    frequency_table = Counter(word.lower() for word in word_tokenize(text) if word.lower() not in stop_words and word.isalpha())

    sentence_scores = {sentence: sum(frequency_table[word.lower()] for word in word_tokenize(sentence) if word.lower() in frequency_table) for sentence in sentences}

    summary_sentences = heapq.nlargest(3, sentence_scores, key=sentence_scores.get)
    return ' '.join(summary_sentences)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [49]:
# Abstractive
def abstraktive_zusammenfassung_bart(text, model_name='facebook/bart-large-cnn'):
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=200, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [66]:
# Rouge
def bewerte_mit_rouge(produzierte_zusammenfassung, referenz_zusammenfassung):
    def formatiere_rouge_ergebnisse(rouge_ergebnisse):
        ergebnis_strings = []
        for score in rouge_ergebnisse:
            for metric, values in score.items():
                ergebnis_strings.append(f"{metric}: Precision: {values['p']:.4f}, Recall: {values['r']:.4f}, F1-Score: {values['f']:.4f}")
        return "\n".join(ergebnis_strings)

    rouge = Rouge()
    scores = rouge.get_scores(produzierte_zusammenfassung, referenz_zusammenfassung)
    return formatiere_rouge_ergebnisse(scores)

In [75]:
# Meteor
def bewerte_mit_meteor(produzierte_zusammenfassung, referenz_zusammenfassung):
    produzierte_zusammenfassung_tokens = word_tokenize(produzierte_zusammenfassung)
    referenz_zusammenfassung_tokens = word_tokenize(referenz_zusammenfassung)
    score = meteor_score([referenz_zusammenfassung_tokens], produzierte_zusammenfassung_tokens)
    return str(score)


In [52]:
# def bewerte_mit_bleu(produzierte_zusammenfassung, referenz_zusammenfassung):
#     referenz = [referenz_zusammenfassung.split()]
#     kandidat = produzierte_zusammenfassung.split()
#     score = sentence_bleu(referenz, kandidat)
#     return score

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
# Einlesen der PDFs
file_path = '/content/drive/My Drive/PDFs/'

Apple =  lese_pdf_datei(file_path + "Apple.pdf")
ref_Apple =  lese_pdf_datei(file_path + "Ref_Apple.pdf")

Bolt =  lese_pdf_datei(file_path + "Bolt.pdf")
ref_Bolt =  lese_pdf_datei(file_path + "Ref_Bolt.pdf")

Discord =  lese_pdf_datei(file_path + "Discord.pdf")
ref_Discord =  lese_pdf_datei(file_path + "Ref_Discord.pdf")


In [55]:
# Druchführen Abstractive

ab_Apple = abstraktive_zusammenfassung_bart(Apple)
ab_Bolt = abstraktive_zusammenfassung_bart(Bolt)
ab_Discord = abstraktive_zusammenfassung_bart(Discord)


In [56]:
# Druchführen Extractive

ex_Apple = extraktive_zusammenfassung(Apple)
ex_Bolt = extraktive_zusammenfassung(Bolt)
ex_Discord = extraktive_zusammenfassung(Discord)

In [77]:
# Bewertung Extracitve

ro_ex_Apple = bewerte_mit_rouge(ex_Apple, ref_Apple)
ro_ex_Bolt = bewerte_mit_rouge(ex_Bolt, ref_Bolt)
ro_ex_Discord = bewerte_mit_rouge(ex_Discord, ref_Discord)

me_ex_Apple = bewerte_mit_meteor(ex_Apple, ref_Apple)
me_ex_Bolt = bewerte_mit_meteor(ex_Bolt, ref_Bolt)
me_ex_Discord = bewerte_mit_meteor(ex_Discord, ref_Discord)

In [78]:
# Bewertung Abstarctive

ro_ab_Apple = bewerte_mit_rouge(ab_Apple, ref_Apple)
ro_ab_Bolt = bewerte_mit_rouge(ab_Bolt, ref_Bolt)
ro_ab_Discord = bewerte_mit_rouge(ab_Discord, ref_Discord)

me_ab_Apple = bewerte_mit_meteor(ab_Apple, ref_Apple)
me_ab_Bolt = bewerte_mit_meteor(ab_Bolt, ref_Bolt)
me_ab_Discord = bewerte_mit_meteor(ab_Discord, ref_Discord)

In [71]:
# Rouge Extractive
print("Apple Extractive: \n" + ro_ex_Apple)
print("Bolt Extractive:  \n" + ro_ex_Bolt)
print("Discord Extractive:  \n" + ro_ex_Bolt)

Apple Extractive: 
rouge-1: Precision: 0.4608, Recall: 0.1418, F1-Score: 0.2168
rouge-2: Precision: 0.2963, Recall: 0.0649, F1-Score: 0.1065
rouge-l: Precision: 0.4461, Recall: 0.1373, F1-Score: 0.2099
Bolt Extractive:  
rouge-1: Precision: 0.3363, Recall: 0.3918, F1-Score: 0.3619
rouge-2: Precision: 0.1564, Recall: 0.1958, F1-Score: 0.1739
rouge-l: Precision: 0.3274, Recall: 0.3814, F1-Score: 0.3524
Discord Extractive:  
rouge-1: Precision: 0.3363, Recall: 0.3918, F1-Score: 0.3619
rouge-2: Precision: 0.1564, Recall: 0.1958, F1-Score: 0.1739
rouge-l: Precision: 0.3274, Recall: 0.3814, F1-Score: 0.3524


In [74]:
# Rouge Abstractive
print("Apple Abstractive:  \n" + ro_ab_Apple)
print("Bolt Abstractive:  \n" + ro_ab_Bolt)
print("Discord Abstractive:  \n" + ro_ab_Discord)

Apple Abstractive:  
rouge-1: Precision: 0.4286, Recall: 0.0271, F1-Score: 0.0511
rouge-2: Precision: 0.0851, Recall: 0.0027, F1-Score: 0.0052
rouge-l: Precision: 0.4048, Recall: 0.0256, F1-Score: 0.0482
Bolt Abstractive:  
rouge-1: Precision: 0.2927, Recall: 0.1237, F1-Score: 0.1739
rouge-2: Precision: 0.0408, Recall: 0.0140, F1-Score: 0.0208
rouge-l: Precision: 0.2439, Recall: 0.1031, F1-Score: 0.1449
Discord Abstractive:  
rouge-1: Precision: 0.5000, Recall: 0.1127, F1-Score: 0.1839
rouge-2: Precision: 0.0847, Recall: 0.0144, F1-Score: 0.0246
rouge-l: Precision: 0.4583, Recall: 0.1033, F1-Score: 0.1686


Extraktive Methode: Bei der extraktiven Methode sind Precision, Recall und F1-Scores für "Bolt" und "Discord" höher als für "Apple", was darauf hindeutet, dass die Zusammenfassungen für "Bolt" und "Discord" näher am Originaltext liegen.
Abstraktive Methode: Die Scores sind generell niedriger, insbesondere bei ROUGE-2, was aufgrund der Neuparaphrasierung der Inhalte in abstraktiven Zusammenfassungen zu erwarten ist.

In [79]:
# Meteor Extractive
print("Apple Extractive:  \n" + me_ex_Apple)
print("Bolt Extractive:  \n" + me_ex_Bolt)
print("Discord Extractive:  \n" + me_ex_Bolt)

Apple Extractive:  
0.10472632806724269
Bolt Extractive:  
0.2797713529856387
Discord Extractive:  
0.2797713529856387


In [80]:
# Meteor Abstractive
print("Apple Abstractive:  \n" + me_ab_Apple)
print("Bolt Abstractive:  \n" + me_ab_Bolt)
print("Discord Abstractive:  \n" + me_ab_Discord)

Apple Abstractive:  
0.016847849739817986
Bolt Abstractive:  
0.07444168734491315
Discord Abstractive:  
0.07254558491023008


Extraktive Methode: Die METEOR-Scores für "Bolt" und "Discord" sind deutlich höher als für "Apple", was darauf hindeutet, dass diese Zusammenfassungen dem Referenzinhalt näher kommen.
Abstraktive Methode: Die Scores sind hier deutlich niedriger, was darauf hindeutet, dass die generierten Zusammenfassungen weniger genau im Vergleich zur Referenzzusammenfassung sind.

Gesamtbewertung:
Die extraktiven Methoden scheinen eine höhere Genauigkeit zu haben, gemessen an den ROUGE- und METEOR-Scores, verglichen mit den abstraktiven Methoden.
Die abstraktiven Methoden zeigen generell niedrigere Scores, was darauf hindeutet, dass die generierten Zusammenfassungen weniger genau mit der Referenzzusammenfassung übereinstimmen. Dies könnte auf die Neuparaphrasierung und den veränderten Kontext der Inhalte in abstraktiven Zusammenfassungen zurückzuführen sein.